In [45]:
--Torch Playground

In [56]:
require 'torch'
require 'paths'
require 'math'
require 'nn'
require 'optim'
require 'gnuplot'

In [57]:
--Load the datasets

In [121]:
mnist = {}
mnist.path_remote = 'https://s3.amazonaws.com/torch7/data/mnist.t7.tgz'
mnist.path_dataset = 'mnist.t7'
--mnist.path_trainset = paths.concat(mnist.path_dataset,'train_32x32.t7')
--mnist.path_testset = paths.concat(mnist.path_dataset,'test_32x32.t7')
mnist.path_trainset = 'train_32x32.t7'
mnist.path_testset  = 'test_32x32.t7'

In [122]:
print(mnist)

{
  path_dataset : mnist.t7
  path_remote : https://s3.amazonaws.com/torch7/data/mnist.t7.tgz
  path_trainset : train_32x32.t7
  path_testset : test_32x32.t7
}


In [123]:
function mnist.download()
    if not paths.filep(mnist.path_trainset) or not paths.filep(mnist.path_testset) then
        local remote = mnist.path_remote
        local tar = paths.basename(remote)
        os.execute('wget '..remote..';'..'tar xvf'..tar..';rm '..tar)
    end
end

In [129]:
function mnist.loadTrainSet(maxLoad, geometry)
    return mnist.loadDataset(mnist.path_trainset, maxLoad, geometry)
end

In [130]:
function mnist.loadTestSet(maxLoad, geometry)
    return mnist.loadDataset(mnist.path_testset, maxLoad, geometry)
end

In [131]:
print(mnist.path_dataset)
--local f = torch.load(fileName, 'ascii')
--mnist.download()
data = mnist.loadTrainSet(opt.train_size, {32,32})

mnist.t7	
train_32x32.t7	


<mnist> loading only8000examples	
<mnist> done	


In [143]:
print(data)

{
  data : DoubleTensor - size: 8000x1x32x32
  normalize : function: 0x4114d238
  labels : ByteTensor - size: 8000
  normalizeGlobal : function: 0x4114d2e0
  size : function: 0x4114d300
}


In [132]:
function mnist.loadDataset(fileName, maxLoad)
    --mnist.download()
    
    print(fileName)
    local f = torch.load(fileName, 'ascii')
    local data = f.data:type(torch.getdefaulttensortype())
    local labels = f.labels
    
    local nExample = f.data:size(1)
    if maxLoad and maxLoad > 0 and maxLoad < nExample then
        nExample = maxLoad
        print('<mnist> loading only' .. nExample .. 'examples')
    end
    data = data[{{1,nExample},{},{},{}}]
    labels = labels[{{1,nExample}}]
    print('<mnist> done')
    
    local dataset = {}
    dataset.data = data
    dataset.labels = labels
    
    function dataset:normalize(mean_, std_)
        local mean = mean_ or data:view(data:size(1),-1):mean(1)
        local std = std_ or data:view(data:size(1),-1):std(1,true)
        for i=1,data:size(1) do
            data[i]:add(-mean[1][i])
            if std[1][i]>0 then
                tensor:select(2,i):mul(1/std[1][i])
            end
        end
        return mean, std
    end
    
    function dataset:normalizeGlobal(mean_,std_)
        local std = std_ or data:std()
        local mean = mean_ or data:mean()
        data:add(-mean)
        data:mul(1/std)
        return mean, std
    end
    
    function dataset:size()
        return nExample
    end
    
    local labelvector = torch.zeros(10)
    
    setmetatable(dataset, {__index = function(self, index)
                            local input = self.data[index]
                            local class = self.labels[index]
                            local label = labelvector:zero()
                            label[class] = 1
                            local example = {input, label}
                                    return example
            
        end})
    
    return dataset
end

In [133]:
opt = {}  -- these options are used throughout
opt.train_size = 8000 -- set to 0 or 60000 to use all 60000 training data
opt.test_size = 0  --0 means load all data
torch.manualSeed(1) -- fix random seed so program runs the same every time
-- NOTE: see below for optimState, storing optimiser settings

--mnist.download() --download dataset if not ready then

In [134]:
--loading and saving the binary and ascii files...

--[[
t = torch.Tensor(10)
torch.save('test-binary.t7', t)
t2 = torch.load('test-binary.t7')
torch.save('test-ascii.t7', t, 'ascii')
t3 = torch.load('test-ascii.t7', 'ascii')
print(t:eq(t2):all())
print(t:eq(t3):all())
]]--

In [135]:
opt.train_size

8000	


In [136]:
data = mnist.loadTrainSet(opt.train_size, {32,32})

train_32x32.t7	


<mnist> loading only8000examples	
<mnist> done	


In [137]:
--load dataset using dataset-mnist.lua into tensors(first dim of data/labels range over data)
function load_dataset(train_or_test, count)
    --load
    local data
    if train_or_test == 'train' then
        data = mnist.loadTrainSet(count, {32,32})
    else
        data = mnist.loadTestSet(count,{32,32})
    end
    
    --shuffle the dataset
    local shuffled_indices = torch.randperm(data.data:size(1)):long()
    --creates a shuffled *copy* with a new storage
    data.data = data.data:index(1, shuffled_indices):squeeze()
    data.labels = data.labels:index(1, shuffled_indices):squeeze()
    -- TODO: (optional) UNCOMMENT to display a training example
    -- for more, see torch gnuplot package documentation:
    -- https://github.com/torch/gnuplot#plotting-package-manual-with-gnuplot
    --gnuplot.imagesc(data.data[10])
    
    --vectorize each 2D data point into 1D
    data.data = data.data:reshape(data.data:size(1),32*32)
    
    print('--------------------------------')
    print(' loaded dataset "' .. train_or_test .. '"')
    print('inputs', data.data:size())
    print('targets', data.labels:size())
    print('--------------------------------')
    
    return data
end

train = load_dataset('train', opt.train_size)
test = load_dataset('test',opt.test_size)


train_32x32.t7	


<mnist> loading only8000examples	
<mnist> done	


--------------------------------	
 loaded dataset "train"	
inputs	
 8000
 1024
[torch.LongStorage of size 2]

targets	
 8000
[torch.LongStorage of size 1]

--------------------------------	
test_32x32.t7	


<mnist> done	


--------------------------------	
 loaded dataset "test"	
inputs	
 10000
  1024
[torch.LongStorage of size 2]

targets	
 10000
[torch.LongStorage of size 1]

--------------------------------	


In [144]:
---------------------------------------------------------------------------------------
-- Practical 3 - Learning to use different optimizers with logistic regression
--
-- to run: th -i practical3.lua
-- or:     luajit -i practical3.lua
---------------------------------------------------------------------------------------

require 'torch'
require 'math'
require 'nn'
require 'optim'
require 'gnuplot'

------------------------------------------------------------------------------
-- INITIALIZATION AND DATA
------------------------------------------------------------------------------

torch.manualSeed(1)    -- fix random seed so program runs the same every time

-- TODO: play with these optimizer options for the second handin item, as described in the writeup
-- NOTE: see below for optimState, storing optimiser settings
local opt = {}         -- these options are used throughout
opt.optimization = 'sgd'
opt.batch_size = 3
opt.train_size = 8000  -- set to 0 or 60000 to use all 60000 training data
opt.test_size = 0      -- 0 means load all data
opt.epochs = 2         -- **approximate** number of passes through the training data (see below for 
                       --the `iterations` variable, which is calculated from this)

-- NOTE: the code below changes the optimization algorithm used, and its settings
local optimState       -- stores a lua table with the optimization algorithm's settings, 
                       --and state during iterations

local optimMethod      -- stores a function corresponding to the optimization routine
-- remember, the defaults below are not necessarily good
if opt.optimization == 'lbfgs' then
  optimState = {
    learningRate = 1e-1,
    maxIter = 2,
    nCorrection = 10
  }
  optimMethod = optim.lbfgs
elseif opt.optimization == 'sgd' then
  optimState = {
    learningRate = 1e-1,
    weightDecay = 0,
    momentum = 0,
    learningRateDecay = 1e-7
  }
  optimMethod = optim.sgd
elseif opt.optimization == 'adagrad' then
  optimState = {
    learningRate = 1e-1,
  }
  optimMethod = optim.adagrad
else
  error('Unknown optimizer')
end

mnist.download()       -- download dataset if not already there

-- load dataset using dataset-mnist.lua into tensors (first dim of data/labels ranges over data)
local function load_dataset(train_or_test, count)
    -- load
    local data
    if train_or_test == 'train' then
        data = mnist.loadTrainSet(count, {32, 32})
    else
        data = mnist.loadTestSet(count, {32, 32})
    end

    -- shuffle the dataset
    local shuffled_indices = torch.randperm(data.data:size(1)):long()
    -- creates a shuffled *copy*, with a new storage
    data.data = data.data:index(1, shuffled_indices):squeeze()
    data.labels = data.labels:index(1, shuffled_indices):squeeze()

    -- TODO: (optional) UNCOMMENT to display a training example
    -- for more, see torch gnuplot package documentation:
    -- https://github.com/torch/gnuplot#plotting-package-manual-with-gnuplot
    --gnuplot.imagesc(data.data[10])

    -- vectorize each 2D data point into 1D
    data.data = data.data:reshape(data.data:size(1), 32*32)

    print('--------------------------------')
    print(' loaded dataset "' .. train_or_test .. '"')
    print('inputs', data.data:size())
    print('targets', data.labels:size())
    print('--------------------------------')

    return data
end

local train = load_dataset('train', opt.train_size)
local test = load_dataset('test', opt.test_size)

------------------------------------------------------------------------------
-- MODEL
------------------------------------------------------------------------------

local n_train_data = train.data:size(1) -- number of training data
local n_inputs = train.data:size(2)     -- number of cols = number of dims of input
local n_outputs = train.labels:max()    -- highest label = # of classes

print(train.labels:max())
print(train.labels:min())

local lin_layer = nn.Linear(n_inputs, n_outputs)
local softmax = nn.LogSoftMax() 
local model = nn.Sequential()
model:add(lin_layer)
model:add(softmax)

------------------------------------------------------------------------------
-- LOSS FUNCTION
------------------------------------------------------------------------------

local criterion = nn.ClassNLLCriterion()

------------------------------------------------------------------------------
-- TRAINING
------------------------------------------------------------------------------

local parameters, gradParameters = model:getParameters()

------------------------------------------------------------------------
-- Define closure with mini-batches 
------------------------------------------------------------------------

local counter = 0
local feval = function(x)
  if x ~= parameters then
    parameters:copy(x)
  end

  -- get start/end indices for our minibatch (in this code we'll call a minibatch a "batch")
  --           ------- 
  --          |  ...  |
  --        ^ ---------<- start index = i * batchsize + 1
  --  batch | |       |
  --   size | | batch |       
  --        v |   i   |<- end index (inclusive) = start index + batchsize
  --          ---------                         = (i + 1) * batchsize + 1
  --          |  ...  |                 (except possibly for the last minibatch, we can't 
  --          --------                   let that one go past the end of the data, so we take a min())
  local start_index = counter * opt.batch_size + 1
  local end_index = math.min(n_train_data, (counter + 1) * opt.batch_size + 1)
  if end_index == n_train_data then
    counter = 0
  else
    counter = counter + 1
  end

  local batch_inputs = train.data[{{start_index, end_index}, {}}]
  local batch_targets = train.labels[{{start_index, end_index}}]
  gradParameters:zero()

  -- In order, these lines compute:
  -- 1. compute outputs (log probabilities) for each data point
  local batch_outputs = model:forward(batch_inputs)
  -- 2. compute the loss of these outputs, measured against the true labels in batch_target
  local batch_loss = criterion:forward(batch_outputs, batch_targets)
  -- 3. compute the derivative of the loss wrt the outputs of the model
  local dloss_doutput = criterion:backward(batch_outputs, batch_targets) 
  -- 4. use gradients to update weights, we'll understand this step more next week
  model:backward(batch_inputs, dloss_doutput)

  -- optim expects us to return
  --     loss, (gradient of loss with respect to the weights that we're optimizing)
  return batch_loss, gradParameters
end
  
------------------------------------------------------------------------
-- OPTIMIZE: FIRST HANDIN ITEM
------------------------------------------------------------------------
local losses = {}          -- training losses for each iteration/minibatch
local epochs = opt.epochs  -- number of full passes over all the training data
local iterations = epochs * math.ceil(n_train_data / opt.batch_size) -- integer number of minibatches to process
-- (note: number of training data might not be divisible by the batch size, so we round up)

-- In each iteration, we:
--    1. call the optimization routine, which
--      a. calls feval(parameters), which
--          i. grabs the next minibatch
--         ii. returns the loss value and the gradient of the loss wrt the parameters, evaluated on the minibatch
--      b. the optimization routine uses this gradient to adjust the parameters so as to reduce the loss.
--    3. then we append the loss to a table (list) and print it
for i = 1, iterations do
  -- optimMethod is a variable storing a function, either optim.sgd or optim.adagrad or ...
  -- see documentation for more information on what these functions do and return:
  --   https://github.com/torch/optim
  -- it returns (new_parameters, table), where table[0] is the value of the function being optimized
  -- and we can ignore new_parameters because `parameters` is updated in-place every time we call 
  -- the optim module's function. It uses optimState to hide away its bookkeeping that it needs to do
  -- between iterations.
  local _, minibatch_loss = optimMethod(feval, parameters, optimState)

  -- Our loss function is cross-entropy, divided by the number of data points,
  -- therefore the units (units in the physics sense) of the loss is "loss per data sample".
  -- Since we evaluate the loss on a different minibatch each time, the loss will sometimes 
  -- fluctuate upwards slightly (i.e. the loss estimate is noisy).
  if i % 10 == 0 then -- don't print *every* iteration, this is enough to get the gist
      print(string.format("minibatches processed: %6s, loss = %6.6f", i, minibatch_loss[1]))
  end
  -- TIP: use this same idea of not saving the test loss in every iteration if you want to increase speed.
  -- Then you can get, 10 (for example) times fewer values than the training loss. If you do this,
  -- you just have to be careful to give the correct x-values to the plotting function, rather than
  -- Tensor{1,2,...,#losses}. HINT: look up the torch.linspace function, and note that torch.range(1, #losses)
  -- is the same as torch.linspace(1, #losses, #losses).

  losses[#losses + 1] = minibatch_loss[1] -- append the new loss
end

-- TODO: for the first handin item, evaluate test loss above, and add to the plot below
--       see TIP/HINT above if you want to make the optimization loop faster

-- Turn table of losses into a torch Tensor, and plot it
gnuplot.plot({
  torch.range(1, #losses),        -- x-coordinates for data to plot, creates a tensor holding {1,2,3,...,#losses}
  torch.Tensor(losses),           -- y-coordinates (the training losses)
  '-'})

------------------------------------------------------------------------------
-- TESTING THE LEARNED MODEL: 2ND HANDIN ITEM
------------------------------------------------------------------------------

local logProbs = model:forward(test.data)
classProbabilities = torch.exp(logProbs)
_, classPredictions = torch.max(classProbabilities, 2)
-- classPredictions holds predicted classes from 1-10

-- TODO: compute test classification error here for the second handin item
-- classPredictions holds predicted classes from 1-10
print(#classPredictions)
print(#test.labels)
local correctCount = 0
for i = 1, classPredictions:size(1) do
  if(classPredictions[i][1] == test.labels[i]) then  correctCount = correctCount + 1 end
end
-- TODO: compute test classification error here for the second handin item
--print('correctCount=',correctCount)--
print('classification error=',(1-correctCount/classPredictions:size(1)))

train_32x32.t7	


<mnist> loading only8000examples	
<mnist> done	


--------------------------------	
 loaded dataset "train"	
inputs	
 8000
 1024
[torch.LongStorage of size 2]

targets	
 8000
[torch.LongStorage of size 1]

--------------------------------	
test_32x32.t7	


<mnist> done	


--------------------------------	
 loaded dataset "test"	
inputs	
 10000
  1024
[torch.LongStorage of size 2]

targets	
 10000
[torch.LongStorage of size 1]

--------------------------------	
10	
1	


...

minibatches processed:   5310, loss = 0.000000	


minibatches processed:   5320, loss = 0.000000	


minibatches processed:   5330, loss = 18459.117386	



 10000
     1
[torch.LongStorage of size 2]


 10000
[torch.LongStorage of size 1]



classification error=	0.1764	


In [149]:
--classPredictions

In [16]:
--Simple Example derrivative

In [29]:
require 'torch'
require 'optim'

In [24]:
--------------------------------------------------------------------------------
-- Simple example of minimizing a function,
--    f(x) = 1/2 x^2 + x sin(x)
-- with derivative
--    f'(x) = x + x cos(x) + sin(x).
--
-- Here's a plot:
-- http://www.wolframalpha.com/input/?i=plot+0.5*x%5E2+%2B+x+sin%28x%29+from+-2+to+4
--
--------------------------------------------------------------------------------

In [35]:
-- preallocate space for gradient, which is 1-dim and 1 element
-- (since f is univariate)
grad = torch.Tensor{0}
print(grad)

 0
[torch.DoubleTensor of size 1]



In [36]:
-- return function's value and the gradient, at a given point x_vec
function feval(x_vec)
    --note: x_vec is a Tensor of 1-dim and size 1, so
    --we get its one and only element:
    local x = x_vec[1]
    --compute and return func val(scalar) and gradient (Tensor)
    f = 0.5*x^2 + x*torch.sin(x)
    grad[1] = x+torch.sin(x) + x*torch.cos(x)
    return f, grad
end

In [37]:
--where to start the algorithm(usually random but here we wont since its demo)
--Note: Try a few starting points, using the plot for pointers
x = torch.Tensor{5}

--optim functions use this table for bookkeeping and for reading options
state = {learningRate = 1e-2}


In [40]:
--stop when the gradient is close to 0 or after many iteration
local iter = 0
while true do
    -- optim has multiple functions, such as adagrad, sgd, lbfgs, and others
    -- see documentation for more details
    optim.adagrad(feval, x, state)
    
    -- gradient norm is SOMETIMES a good measure of how close we are to the optimum, but often not.
    -- the issue is that we'd stop at points like x=0 for x^3
    if grad:norm() < 0.005 or iter > 50000 then
        break
    end
    iter = iter + 1
end

print('iter='..iter)
print(string.format("%.6f",x[1]))

iter=4285	
3.677549	


nil	


In [150]:
-- iris data sets 

In [153]:
require 'csv'
require 'nn'

In [154]:
--prepare data set
dataset = {}
size = 0

In [155]:
ans = {setosa=1,versicolor=2,virginica=3}

In [156]:
local f = csv.open('iris.csv')
for cols in f:lines() do
    local input = torch.Tensor(4)
    local output = torch.Tensor(3)
    for i,v in ipairs(cols) do
        if i>=2 and i<=5 then
            input[i-1] = v
        end
        if i==6 then
            for j=1,3 do
                output[j] = j==ans[v] and 1 or 0
            end
        end
        print(input,output)
    end
    size = size+1
    dataset[size] = {input,output}
end

function dataset:size() return size end

--make model
nhidden = 5

model = nn.Sequential();
model:add(nn.Linear(4,nhidden))
model:add(nn.Tanh())
model:add(nn.Linear(nhidden,3))
model:add(nn.SoftMax())

print(model)

criterion = nn.MSECriterion()
trainer = nn.StochasticGradient(model,criterion)
trainer.learningRate = 0.01
trainer.maxIteration = 1000
trainer:train(dataset)

test1_in = torch.Tensor(4)
test1_in[1] = 6.7
test1_in[2] = 2.5
test1_in[3] = 5.8
test1_in[4] = 1.8

test1_out = model:forward(test1_in)
print(test1_out)

1e-310 *
 6.9202
 6.9202
 0.0004
 0.0000
[torch.DoubleTensor of size 4]



1e-310 *
 6.9202
 6.9202
 0.0000
[torch.DoubleTensor of size 3]

 5.1000
 0.0000
 0.0000
 0.0000
[torch.DoubleTensor of size 4]

1e-310 *
 6.9202
 6.9202
 0.0000
[torch.DoubleTensor of size 3]

 5.1000
 3.5000
 0.0000
 0.0000
[torch.DoubleTensor of size 4]

1e-310 *
 6.9202
 6.9202
 0.0000
[torch.DoubleTensor of size 3]

 5.1000
 3.5000
 1.4000
 0.0000
[torch.DoubleTensor of size 4]

1e-310 *
 6.9202
 6.9202
 0.0000
[torch.DoubleTensor of size 3]



 5.1000
 3.5000
 1.4000
 0.2000
[torch.DoubleTensor of size 4]

1e-310 *
 6.9202
 6.9202
 0.0000
[torch.DoubleTensor of size 3]

 5.1000
 3.5000
 1.4000
 0.2000
[torch.DoubleTensor of size 4]

 1
 0
 0
[torch.DoubleTensor of size 3]

1e-310 *
 6.9202
 6.9202
 0.0000
 0.0000
[torch.DoubleTensor of size 4]

1e-310 *
 6.9202
 6.9202
 0.0000
[torch.DoubleTensor of size 3]

 4.9000
 0.0000
 0.0000
 0.0000
[torch.DoubleTensor of size 4]

1e-310 *
 6.9202
 6.9202
 0.0000
[torch.DoubleTensor of size 3]

 4.9000
 3.0000
 0.0000
 0.0000
[torch.DoubleTensor of size 4]



1e-310 *
 6.9202
 6.9202
 0.0000
[torch.DoubleTensor of size 3]

 4.9000
 3.0000
 1.4000
 0.0000
[torch.DoubleTensor of size 4]

1e-310 *
 6.9202
 6.9202
 0.0000
[torch.DoubleTensor of size 3]

 4.9000
 3.0000
 1.4000
 0.2000
[torch.DoubleTensor of size 4]

1e-310 *
 6.9202
 6.9202
 0.0000
[torch.DoubleTensor of size 3]

 4.9000
 3.0000
 1.4000
 0.2000
[torch.DoubleTensor of size 4]

 1
 0
 0
[torch.DoubleTensor of size 3]

1e-310 *
 6.9202
 6.9202
 0.0000
 0.0000
[torch.DoubleTensor of size 4]

1e-310 *
 6.9202
 6.9202
 0.0000
[torch.DoubleTensor of size 3]



 4.7000
 0.0000
 0.0000
 0.0000
[torch.DoubleTensor of size 4]

1e-310 *
 6.9202
 6.9202
 0.0000
[torch.DoubleTensor of size 3]

 4.7000
 3.2000
 0.0000
 0.0000
[torch.DoubleTensor of size 4]

1e-310 *
 6.9202
 6.9202
 0.0000
[torch.DoubleTensor of size 3]

 4.7000
 3.2000
 1.3000
 0.0000
[torch.DoubleTensor of size 4]

1e-310 *
 6.9202
 6.9202
 0.0000
[torch.DoubleTensor of size 3]

 4.7000
 3.2000
 1.3000
 0.2000
[torch.DoubleTensor of size 4]

1e-310 *
 6.9202
 6.9202
 0.0000
[torch.DoubleTensor of size 3]

 4.7000
 3.2000
 1.3000
 0.2000
[torch.DoubleTensor of size 4]

 1
 0
 0
[torch.DoubleTensor of size 3]



...

# current error = 0.012660554340139	


# current error = 0.012661332328854	


# current error = 0.012662114513358	


# current error = 0.012662900888141	


# current error = 0.012663691447444	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 0.012663691447444	
 0.0001
 0.0327
 0.9672
[torch.DoubleTensor of size 3]



In [157]:
------------------------------------------------------------------------------------------------------------

In [158]:
----Practical 4, Oxford

In [159]:
--load the csv data file

In [192]:
require 'torch'
require 'nn'
require 'math'
require 'optim'
require 'gnuplot'

-- Similar to mnist-loader from last practical, loads IRIS dataset train/test split
-- has load_train function that returns table like { inputs = ... , targets = ... }, a tensor for inputs and targets.
-- has classname_to_index function that returns a table mapping class number (1, 2, 3) to the 3 flower names

local loader = {}

local lookup = {"Iris-setosa", "Iris-versicolor", "Iris-virginica"}
local classname_to_index = {}
for k,v in pairs(lookup) do
    classname_to_index[v] = k
end
function loader.classname_to_index(name)
  return classname_to_index[name]
end

function load_data()
  -- load
  local data = {}
  data.inputs = {}
  data.targets = {}
  data.targets_by_name = {}

  local f = torch.DiskFile("iris.data.csv", "r")
  f:quiet()

  local line =  f:readString("*l")
  while line ~= '' do
      f1, f2, f3, f4, class_name = string.match(line, '([^,]+),([^,]+),([^,]+),([^,]+),([^,]+)')
      data.inputs[#data.inputs + 1] = {tonumber(f1), tonumber(f2), tonumber(f3), tonumber(f4)}
      data.targets[#data.targets + 1] = loader.classname_to_index(class_name)
      data.targets_by_name[#data.targets_by_name + 1] = class_name
      line = f:readString("*l")
  end

  data.inputs = torch.Tensor(data.inputs)
  data.targets = torch.Tensor(data.targets)

  -- shuffle the dataset
  local shuffled_indices = torch.randperm(data.inputs:size(1)):long()
  -- creates a shuffled *copy*, with a new storage
  data.inputs = data.inputs:index(1, shuffled_indices):squeeze()
  data.targets = data.targets:index(1, shuffled_indices):squeeze()

  print('--------------------------------')
  print('Loaded. Sizes:')
  print('inputs', data.inputs:size())
  print('targets', data.targets:size())
  print('--------------------------------')

  return data
end

--return loader


In [193]:
data = load_data()

--------------------------------	
Loaded. Sizes:	
inputs	
 150
   4
[torch.LongStorage of size 2]

targets	
 150
[torch.LongStorage of size 1]

--------------------------------	


In [203]:
--create model
function create_model(opt)
  ------------------------------------------------------------------------------
  -- MODEL
  ------------------------------------------------------------------------------
  local n_inputs = 4
  local embedding_dim = 2
  local n_classes = 3

  -- OUR MODEL:
  --     linear -> sigmoid/requ -> linear -> softmax
  local model = nn.Sequential()
  model:add(nn.Linear(n_inputs, embedding_dim))

  if opt.nonlinearity_type == 'requ' then
    model:add(nn.ReQU())
  elseif opt.nonlinearity_type == 'sigmoid' then
    model:add(nn.Sigmoid())
  else
    error('undefined nonlinearity_type ' .. tostring(opt.nonlinearity_type))
  end 

  model:add(nn.Linear(embedding_dim, n_classes))
  model:add(nn.LogSoftMax())

  ------------------------------------------------------------------------------
  -- LOSS FUNCTION
  ------------------------------------------------------------------------------
  local criterion = nn.ClassNLLCriterion()

  return model, criterion
end

return create_model


In [207]:
local ReQU = torch.class('nn.ReQU','nn.Module')

function ReQU:updateOutput(input)
    self.output:resizeAs(input):copy(input)
    return self.output
end

function ReQU:updateGradInput(input, gradOutput)
    self.gradInput:resizeAs(gradOutput):copy(gradOutput)
    return self.gradInput
end

/usr/local/share/lua/5.1/torch/init.lua:75: class nn.ReQU has been already assigned a parent class

stack traceback:
	[C]: in function 'newmetatable'
	/usr/local/share/lua/5.1/torch/init.lua:75: in function 'class'
	[string "local ReQU = torch.class('nn.ReQU','nn.Module..."]:1: in main chunk
	[C]: in function 'xpcall'
	/home/rbhat/torch/install/share/lua/5.1/itorch/main.lua:179: in function </home/rbhat/torch/install/share/lua/5.1/itorch/main.lua:143>
	/home/rbhat/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	/home/rbhat/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	/home/rbhat/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	/home/rbhat/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	/home/rbhat/torch/install/share/lua/5.1/itorch/main.lua:350: in main chunk
	[C]: in function 'require'
	[string "arg={'/run/user/1000/jupyter/kernel-ff850527-..."]:1: in main chunk: 

In [205]:
function train(opt, data)
    
    local model, criterion = create_model(opt)
    local params, grads    = model:getParameters()
    
    --re-initialize weights
    params:uniform(-0.01,0.01)
    if opt.nonlinearity_type == 'requ' then
        for _, lin in pairs(model:findModules('nn.Linear')) do
            lin.bias:add(0.5)
        end
    end
    
    --return loss, grad
    local feval = function (x)
        if x ~= params then
            params:copy(x)
        end
        grads:zero()
        
        --forward
        local outputs = model:forward(data.inputs)
        local loss    = criterion:forward(outputs, data.targets)
        
        --backward
        local dloss_doutput = criterion:backward(outputs, data.targets)
        model:backward(data.inputs, dloss_doutput)
        
        return loss, grads
    end
    
    --optimization loop
    local losses = {}
    local optim_state = {learningRate = 1e-1}
    
    for i=1, opt.training_iterations do
        local _,loss = optim.adagrad(feval,params,optim_state)
        losses[#losses+1] = loss[1] --append the new loss
        
        if i % opt.print_every == 0 then
            print(string.format("iteration %4d, loss=%6.6f", i, loss[1]))
        end
    end
    
    return model, losses
end

In [206]:
torch.manualSeed(1)
local data = load_data()

local opt = {
  nonlinearity_type = 'sigmoid',
  training_iterations = 150, -- note: the code uses *batches*, not *minibatches*, now.
  print_every = 25,          -- how many iterations to skip between printing the loss
}

-- train sigmoid and requ versions
model_sigmoid, losses_sigmoid = train(opt, data)
-- TODO: uncomment once you implement requ
--opt.nonlinearity_type = 'requ'
--model_requ, losses_requ = train(opt, data)


--------------------------------------------------------
-- EVALUATION STUFF: YOU CAN IGNORE ALL THIS CODE
-- NOTE: though we don't have a test set, but we'll plot the two training loss curves
-- We won't know if we overfit, but we can see how flexible our model is.

-- plot
gnuplot.figure()
gnuplot.plot({'sigmoid',
  torch.range(1, #losses_sigmoid), -- x-coordinates
  torch.Tensor(losses_sigmoid),    -- y-coordinates
  '-'}
  -- TODO: uncomment when you implement requ
  -- , {'requ',
  -- torch.range(1, #losses_requ),    -- x-coordinates
  -- torch.Tensor(losses_requ),       -- y-coordinates
  -- '-'}
  )

models = { 
    --requ = model_requ,  -- TODO: uncomment once you implement requ
    sigmoid = model_sigmoid 
}
for model_name, model in pairs(models) do
  -- classification error on train set
  local log_probs = model:forward(data.inputs)
  local _, predictions = torch.max(log_probs, 2)
  print(string.format('# correct for %s:', model_name))
  print(torch.mean(torch.eq(predictions:long(), data.targets:long()):double()))

  -- classification region in one slice (cf. Figure 1 scatterplots in writeup)
  -- not pretty, but the best we can do without hacking away at gnuplot or using another library
  local f1 = 4 -- feature on first axis
  local f2 = 3 -- feature on second axis
  local size = 60  -- resolution
  local f1grid = torch.linspace(data.inputs[{{},f1}]:min(), data.inputs[{{},f1}]:max(), size)
  local f2grid = torch.linspace(data.inputs[{{},f2}]:min(), data.inputs[{{},f2}]:max(), size)
  local result = torch.Tensor(size, size)
  local input = data.inputs[1]:clone()
  for i=1,size do
    input[f1] = f1grid[i]
    for j=1,size do
      input[f2] = f2grid[j]
      result[{i,j}] = math.exp(model:forward(input)[1])
    end
  end
  result[1][1] = 0 -- ugly hack to get the right scale
  result[1][2] = 1 -- ugly hack to get the right scale
  gnuplot.figure()
  gnuplot.imagesc(result, model_name)
end

--------------------------------	
Loaded. Sizes:	
inputs	


 150
   4
[torch.LongStorage of size 2]

targets	
 150
[torch.LongStorage of size 1]

--------------------------------	


iteration   25, loss=0.691419	


iteration   50, loss=0.575468	


iteration   75, loss=0.490374	


iteration  100, loss=0.411512	


iteration  125, loss=0.353268	


iteration  150, loss=0.309571	


# correct for sigmoid:	


0.97333333333333	


In [1]:
--Torch LSTM